# Examples for IGM $f(N)$ constraints (v1.1)

In [1]:
%matplotlib inline

In [2]:
# imports
try:
    import seaborn as sns; sns.set_style("white")
except:
    pass

import time
import imp

import bokeh
from bokeh.io import output_notebook, show, hplot, output_file
from bokeh.plotting import figure
from bokeh.models import Range1d

output_notebook()

from pyigm.fN.fnmodel import FNModel
from pyigm.fN.constraints import FNConstraint

from pyigm.fN import tau_eff as teff

pyigm_path = imp.find_module('pyigm')[1]

/Users/xavier/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Loading BokehJS ...

## Load

### Vanilla constraints

In [3]:
fn_file = pyigm_path+'/data/fN/fn_constraints_z2.5_vanilla.fits'
k13r13_file = pyigm_path+'/data/fN/fn_constraints_K13R13_vanilla.fits'
n12_file = pyigm_path+'/data/fN/fn_constraints_N12_vanilla.fits'

In [6]:
#reload(fNc)
all_fN_cs = FNConstraint.from_fitsfile([fn_file,k13r13_file, n12_file])

In [7]:
len(all_fN_cs)

8

### Remove K02

In [8]:
fN_cs = [fN_c for fN_c in all_fN_cs if ((fN_c.ref != 'K02') & (fN_c.ref != 'PW09'))]
fN_dtype = [fc.fN_dtype for fc in fN_cs]
fN_cs

[<FNConstraint: fN_SLLS zeval=2.51004, ref=OPB07>,
 <FNConstraint: LLS_\tlox zeval=2.22584, ref=OPW12>,
 <FNConstraint: MFP_\lmfp zeval=2.44, ref=OPW13>,
 <FNConstraint: teff_\tlya zeval=2.4, ref=K05>,
 <FNConstraint: fN_Lya Forest zeval=2.37, ref=K13R13>,
 <FNConstraint: fN_DLA zeval=2.5, ref=N12>]

## Plot $f(N)$

### $f(N)$ Model

In [9]:
fN_model = FNModel.default_model()

Using P14 spline values to generate a default model
Loading: /Users/xavier/local/Python/pyigm/pyigm/data/fN/fN_spline_z24.fits.gz


### Data dict

In [10]:
data_dict = dict(Nbin=[], fN=[], ref=[], pcolor=[], zeval=[])

In [11]:
clrs = ['blue', 'green', 'red', 'purple']
cnt=0
for fN_c in fN_cs: 
    if fN_c.fN_dtype == 'fN':
        # Length
        ip = range(fN_c.data['NPT'])
        val = np.where(fN_c.data['FN'][ip] > -90)[0]
        npt = len(val)
        # Add in
        if npt > 0:
            ipv = np.array(ip)[val]
            data_dict['Nbin'] += list(np.median(fN_c.data['BINS'][:,ipv],0))
            #xerror = [ fN_c.data['BINS'][1,ipv]-xval, xval-fN_c.data['BINS'][0,ipv] ]
            #yerror = [ fN_c.data['SIG_FN'][1,ipv], fN_c.data['SIG_FN'][0,ipv] ] # Inverted!
            data_dict['fN'] += list(fN_c.data['FN'][ipv])
            data_dict['ref'] += [fN_c.ref]*npt
            data_dict['pcolor'] += [clrs[cnt]]*npt
            data_dict['zeval'] += [fN_c.zeval]*npt
            cnt += 1

In [12]:
plot_source = bokeh.models.ColumnDataSource(data_dict)

In [13]:
if fN_model is not None: 
    xplt = 12.01 + 0.01*np.arange(1100)
    line_dict = dict(xplt=xplt,
                     yplt=fN_model.evaluate(xplt, 2.4).flatten(),
                     ref=['P14']*1100, zeval=[fN_model.zpivot]*1100)
    model_source = bokeh.models.ColumnDataSource(line_dict)

In [14]:
p = bokeh.plotting.figure(tools="reset, hover, tap, resize, wheel_zoom, pan",
                          title="f(N)",
                          x_axis_label="log N_HI",
                          y_axis_label="log f(N)")
#
if fN_model is not None:
    p.line("xplt", "yplt", source=model_source, color='black', line_width=2, legend='P14')
# plot our data
data = p.scatter("Nbin", "fN", 
          source=plot_source, 
          color="pcolor", 
          size=8, legend='data')
# print useful data when you hover
hover = p.select(bokeh.models.HoverTool)
hover.tooltips = {"Ref"    : "@ref",
                  "redshift" : "@zeval"}
#
show(p)

## Plot other constraints

In [15]:
con_dict = dict(xval=[], value=[], error=[], ftype=[], pcolor=[], ref=[], zeval=[])

### $\tau_{\rm eff}$

In [18]:
itau = fN_dtype.index('teff') # Passes back the first one

In [19]:
fN_cs[itau].data['TEFF']

0.19778812

In [20]:
itau = fN_dtype.index('teff')     
con_dict['value'] += [float(fN_cs[itau].data['TEFF'])]
D_A = 1. - np.exp(-1. * con_dict['value'][0])
SIGDA_LIMIT = 0.1  # Allows for systemtics and b-value uncertainty
con_dict['error'] += [np.max([fN_cs[itau].data['SIG_TEFF'], (SIGDA_LIMIT*con_dict['value'][0])])]
con_dict['pcolor'] += ['blue']
con_dict['ref'] += [fN_cs[itau].ref]
con_dict['zeval'] += [fN_cs[itau].zeval]
con_dict['xval'] += [1]
con_dict['ftype'] += ['tau_eff']

### $\ell(X)$

In [21]:
iLLS = fN_dtype.index('LLS') # Passes back the first one
con_dict['value'] += [fN_cs[iLLS].data['LX']]
con_dict['error'] += [fN_cs[iLLS].data['SIG_LX']]
con_dict['pcolor'] += ['green']
con_dict['ref'] += [fN_cs[iLLS].ref]
con_dict['zeval'] += [fN_cs[iLLS].zeval]
con_dict['xval'] += [2]
con_dict['ftype'] += ['l(X)']

### $\lambda_{\rm mfp}$

In [22]:
iMFP = fN_dtype.index('MFP') 
con_dict['value'] += [fN_cs[iMFP].data['MFP']/500]
con_dict['error'] += [fN_cs[iMFP].data['SIG_MFP']/500]
con_dict['pcolor'] += ['red']
con_dict['ref'] += [fN_cs[iMFP].ref]
con_dict['zeval'] += [fN_cs[iMFP].zeval]
con_dict['xval'] += [3]
con_dict['ftype'] += ['MFP/(500 Mpc)']

### Model

In [23]:
x_model = [1,2,3]
y_model = [None]*3

In [24]:
# teff
y_model[0] = teff.lyman_ew(1215.6701*(1+fN_cs[itau].zeval), 
                           fN_cs[itau].zeval+0.1, fN_model, 
                           NHI_MIN=fN_cs[itau].data['NHI_MNX'][0],
                           NHI_MAX=fN_cs[itau].data['NHI_MNX'][1])

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/Users/xavier/local/Python/pyigm/pyigm/fN/tau_eff.py:222: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  restEW = interpolate.splev(lgNval, EW_spline['tck'][idx], der=0)


In [25]:
# l(X)
y_model[1] = fN_model.calculate_lox(fN_cs[iLLS].zeval, 
                                    17.19+np.log10(fN_cs[iLLS].data['TAU_LIM']), NHI_max=22.)

In [26]:
# MFP
y_model[2] = fN_model.mfp(fN_cs[iMFP].zeval).value / 500.

In [27]:
x_model, y_model

([1, 2, 3], [0.19818321669381098, 0.33534482853058251, 0.5142051134453323])

### Plot

In [28]:
con_source = bokeh.models.ColumnDataSource(con_dict)

In [29]:
p = bokeh.plotting.figure(tools="reset, hover, tap, resize, wheel_zoom, pan",
                          title="Integral Constraints",
                          y_axis_label="Value")
# Constraints
p.scatter("xval", "value", source=con_source, color="pcolor", size=7)
err_xs = []
err_ys = []
for x, y, yerr in zip(con_dict['xval'], con_dict['value'], con_dict['error']):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))
p.multi_line(err_xs, err_ys, color='gray', line_width=2)
#
p.text(con_dict['xval'], [0.]*3, con_dict['ftype'], text_color=con_dict['pcolor'],
      text_align="center")
# Model
p.square(x_model, y_model, color='black', fill_color=None, size=10, line_width=2, legend='Model')
# Hover
hover = p.select(bokeh.models.HoverTool)
hover.tooltips = {"Ref"    : "@ref",
                  "Type"   : "@ftype", 
                  "Value"  : "@value", 
                  "Error"  : "@error", 
                  "redshift" : "@zeval"}
# style
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.major_label_text_color = None
p.set(x_range=Range1d(0.5,3.5), y_range=Range1d(0.,0.6))
p.legend.orientation = "top_left"
#
show(p)

/Users/xavier/anaconda/lib/python2.7/site-packages/bokeh/core/properties.py:714: BokehDeprecationWarning: bokeh.models.annotations.orientation was deprecated in Bokeh 0.11; please use Legend.location instead
  super(HasProps, self).__setattr__(name, value)
